# Intro

- This notebook will show how the target works. 
- The point of the ML model in this project is not to predict the share price directly, but to meet the needs of the target in our own trading.

In [67]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go



In [41]:
ticker = "TSLA"
df = yf.download(ticker, period="6mo", interval="1d", progress=False)

In [68]:
def get_plotly(df, ticker):
    fig = go.Figure(data=[go.Candlestick(x=df.index,
                                        open=df["Open"],
                                        high=df["High"],
                                        low=df["Low"],
                                        close=df["Close"])])

    fig.update_layout(
            title=f"{ticker} price",
            xaxis_title="Date",
            yaxis_title="Price",
            autosize=True,
            # showlegend=True,
            xaxis_rangeslider_visible=False,
            margin=dict(l=0, r=10, t=50, b=50)
        )
    return fig


In [60]:
fig = get_plotly(df, ticker)
fig.show()

# The objective
- In 5 days, the price reached +5% before hitting a -5%
- This works with the trading strategy of take 5% profit with a -5% stop loss
- Would require > 50% accuracy to break even
- Change to: reach 6% before hitting -3% stop loss would only require 33% accuracy to break even

In [64]:
df["entry"] = 0
for day in range(1, 6):
    zero_idx = df[df["entry"] == 0].index

    # check if reach 5%, then check -5%
    change_pct = (df["High"].shift(-day) - df["Close"]) / df["Close"]
    df.loc[zero_idx, "entry"] = np.where(change_pct[zero_idx] > 0.05, 1, 0)

    zero_idx = df[df["entry"] == 0].index

    change_pct = (df["Close"] - df["Low"].shift(-day)) / df["Close"]
    df.loc[zero_idx, "entry"] = np.where(change_pct[zero_idx] > 0.05, -1, 0)



In [65]:
df["entry"].value_counts()

entry
-1    54
 1    47
 0    24
Name: count, dtype: int64

In [66]:
fig = get_plotly(df, ticker)

result_pairs = [[-1, 1.05, "bear", "red"],
                [0, 1.00, "neutral", "black"],
                [1, 0.95, "bull", "green"]]

for [flag, factor, name, color] in result_pairs:
    idx = df[df["entry"] == flag].index
    fig.add_trace(
        go.Scatter(
            x=idx,
            y=df["Close"][idx] * factor,
            mode="markers+text",
            marker=dict(color=color, size=8),
            name=name
            )
            )
fig.show()